### **Storing with empty directories**

This notebook will show how to use an empty directory to store objects. Note that empty directories are not a permanent storage option, and can not pass objects between two steps. They only offer the possibility to save objects and use them in the same step.

**First some imports and global settings**

In [ ]:
from hera.workflows import EmptyDirVolume, models, script, Artifact, Parameter, DAG, Steps, Step, NoneArchiveStrategy, Workflow
from hera.shared import global_config

global_config.host = "https://services.eodc.eu/workflows/"
global_config.namespace = "<YOUR NAMESPACE>"
global_config.token = "<YOUR TOKEN>"
global_config.image = "ghcr.io/eodcgmbh/cluster_image:2025.2.0"

**Setting up Volume**

First an empty directory has to be set up as a `Volume`, this can be used in multiple steps. As already mentioned, we can not pass objects in this volume between steps.  

In [ ]:
empty_dir = models.Volume(name="emptydir", empty_dir=models.EmptyDirVolumeSource())

**Writing scripts**

In the `@script` decorator we have to mount the empty directory Volume, we can mount multiple volumes as a list if needed. In this script we download a file from the geosphere datahub, save it in the empty directory and open and read it. 

In [ ]:
@script(volume_mounts=[models.VolumeMount(name="emptydir", mount_path="/mnt/vol/")])
def download_and_use():
    from urllib.request import urlretrieve
    import datetime
    import os
    import xarray as xr

    mount_path = "/mnt/vol/"

    url = f"https://public.hub.geosphere.at/datahub/resources/inca-v1-1h-1km/filelisting/RR/INCAL_HOURLY_RR_202506.nc"
    urlretrieve(url, os.path.join(mount_path, "INCA.nc"))

    ds = xr.open_dataset(os.path.join(mount_path, "INCA.nc")).isel(time=1, x=1, y=1)["RR"].load()
    print(ds.values)

**Creating the Workflow**

In the workflow we have to define that we will use a volume, again multiple volumes can be given in a list.

In [ ]:
with Workflow(
    generate_name="empty-dir-",
    volumes = [empty_dir],
    entrypoint="workflow"
) as w:
    with DAG(name="workflow"):
        download_and_use()

**Submitting the Workflow**

In [ ]:
w.create()